## 计算公式

中轨=N日移动平均线

上轨=中轨+M*N日标准差

下轨=中轨-M*N日标准差

Mid=ma（C，N）

Up=mid+M*std（C，N）

Down=mid-M*std（C，N）

备注：c为收盘价close，ma为移动平均线，std为标准差

## 数学逻辑

简而言之，当M=2时，上轨就是均值加上2倍的标准差，下轨就是均值减两倍的标准差，均值加减2倍的标准差代表的什么？数学或统计学专业的同学都会发现这不就是可信赖区间吗，如果说股价是一个正态分布法则，而2正式取概率值为97.5%的正态分布函数反解的取值，也就是说上轨之上的概率为2.5%，同理下轨之下的概率为2.5%，上轨下轨之间的概率为95%，价格在大概率下是归属于上下轨之间的。

## 趋势交易思想

顺势而为，上涨突破关键位置（上轨）买入，下跌跌破关键位置（下轨）卖出。也就是说强势时买入，弱势时卖出，也就是我们常说的右侧交易。

## 基础版BOLL策略

现在我们来开始确定这两个关键位置，一般来说上方的关键位置就是BOLL上轨，下方的关键位置就是BOLL下轨，中轨均线周期一般取10-30，中轨周期越小一般来说交易越频繁。
我们这里暂时只测试单品种，所以希望交易次数适当多一点取周期为n=15，而我们选取方差倍数m=2.2（2是95%的可信赖区间，2.2大概是98%的可信赖区间），同时由于本策略是一个标准的择时策略而不是选股策略所以我们需要选出一只或几只标的股进行买卖。
而我们希望用BOLL策略去做大盘（A股市场）择时，所以这里我们选用沪深300指数作为标的，同时沪深300也可以作为股指期货品种来交易（本策略为多头策略，对于期货等双向市场需加入空头策略）

## K线过程中突破日线BOLL（重点）

将昨日收盘价高于上轨改成昨天最高价高于上轨也就是说昨日只有存在高于上轨的情况即买入，卖出还是保留收盘价比对下轨，因为我们希望不要轻易被洗出去

In [ ]:
def initialize(context):
    set_benchmark('000300.XSHG')
    set_option('use_real_price',True)
    set_option('order_volume_ratio', 1)
    set_order_cost(OrderCost(open_tax=0, close_tax=0.001, open_commission=0.0003, close_today_commission=0,close_commission=0.0003,min_commission=5), type='stock')
    g.security = '000300.XSHG'   # 可多加股票
    g.M = 15   # 20日均线
    g.N = 2.2    # N
    
def handle_data(context,data):
    sr = attribute_history(g.security,g.M)['close']
    ma20 = sr.mean()                                 # 20日均线
    up = ma20 + g.N * sr.std()
    down = ma20 - g.N * sr.std()
    q=attribute_history(g.security,1)['high'].mean()
    v=attribute_history(g.security,1)['close'].mean()
    p = get_current_data()[g.security].day_open      # 开盘价
    amount = context.portfolio.positions[g.security].total_amount # 总持仓 。或者开仓均线去表示
    cash = context.portfolio.available_cash                       # 可用资金
    
    # 如果p小于down，且没有持仓，则 买入
    # 如果p大于up，且有持仓，则 卖出
    if  q> up and amount == 0:
        order_value(g.security,cash) 
    elif v< down and amount > 0:
        order_target(g.security,0)

<img src="boll.png">